#### Q. 1. Build a ANN model from scratch for predicting best housing selling prices in Boston using three features (i.e. “RM: average number of rooms per dwelling; LSTAT: percentage of population considered lower status: PTRATIO: pupil-teacher ratio by town”) of Boston dataset (Use Sklearn Dataset) by using Stochastic Gradient Descent algorithm for the loss functions: 
- (a) Mean Square Error 
- (b) Huber Loss 
- (c) Squared Epsilon Hinge Loss 
- (i) Plot comparative loss curve for at least 500 epochs.
- (ii) Print comparison of Boston housing selling prices among above mentioned loss functions using bar chart plot and which loss function is providing better housing selling prices among others.
- (iii) Implement above ANN model with Keras Library and verify the above results.

#### Q. 2. Build a ANN model from to recognize breast cancer from Breast Dataset (Use Sklearn Dataset). Use Stochastic gradient descent algorithm to learn model with parameters for α = 0.01 and random parameters of the parameters of the ANN model for loss functions 
- (a) Binary cross entropy 
- (b) Dice Loss 
- (i) Plot comparative loss curve for at least 200 epochs.
- (ii) Print confusion matrix, calculate classification metrics such as precision, recall, f1-score and accuracy and ROUC curve for each loss function.
- (iii) Repeat part (ii) to (iii) using Adam gradient descent algorithm
- (iv) Implement above ANN model with Keras Library and verify the above results. 

#### Q.3. Build a ANN model from scratch to recognize diabetes-from pima-indians-diabetes-database (i.e. https://github.com/duonghuuphuc/keras/tree/master/dataset ). Use Stochastic gradient descent algorithm to learn model with parameters for α = 0.01 and random parameters of the ANN model for loss functions 
- (a) Binary cross entropy 
- (b) Dice Loss 
- (i) Visualize input dataset and Plot comparative loss curve for at least 200 epochs.
- (ii) Print confusion matrix, calculate classification metrics such as precision, recall, f1-score and accuracy and ROUC curve for each loss function.
- (iii) Repeat part (i) to (ii) using Adam gradient descent algorithm
- (iv) Implement above ANN model with Keras Library and verify the above results. 

#### Q. 4. Build a ANN model from scratch to recognize Iris-setosa, Iris -virginica and Iris-versicolor from the Iris Dataset ((Use Sklearn Dataset) which contains four features (length and width of sepals and petals) of 50 samples of three species of Iris (Iris setosa, Iris virginica and Iris versicolor. For implementation, use Stochastic gradient descent algorithm to learn model with parameters for α = 0.01and random parameters of the ANN model for the Softmax loss function 
- (i)Visualize data by boxplot of Sepal Length & Sepal width and Petal Length and width for three IRIS
species. 
- (ii) Plot comparative loss curve for at least 200 epochs.
- (iii) Print confusion matrix, calculate classification metrics such as precision, recall, f1-score and accuracy and ROUC curve
- (iv) Visualize classified data by Scatter plot
- (v) Print confusion matrix, calculate classification metrics such as precision, recall, f1-score and accuracy and ROUC curve for each loss function.
- (vi) Repeat part (ii) to (v) using Adam gradient descent algorithm
- (vii) Implement above ANN model with Keras Library and verify the above results. 

#### Q. 5. Build a ANN model from scratch to recognize human emotion using Facial emotion recognition dataset (FER2013) (https://github.com/gitshanks/fer2013). For implementation, use Stochastic gradient descent algorithm to learn model with parameters for α = 0.01 and random parameters of the ANN model for the Softmax loss function 
- (i)Visualize Facial emotion recognition dataset (FER2013. 
- (ii) Plot comparative loss curve for at least 200 epochs.
- (iii)Print confusion matrix, calculate classification metrics such as precision, recall, f1-score and accuracy and ROUC curve
- (iv) Repeat part (ii) to (iii) using Adam gradient descent algorithm
- (v) Implement above ANN model with Keras Library and verify the above result

In [106]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm
import pandas as pd

In [202]:
def softmax(Input, Derivative = False , dE = None):
    
    if(Derivative):
        Soft = np.array([softmax(Input)])
        der = Soft*(np.identity(len(Input)) - np.transpose(Soft))
        output = np.matmul(np.array([dE]),der)[0]
        return output
    else:
        Input = np.array(Input)
        Numerator = np.exp(Input)
        output = Numerator/sum(Numerator)
        return (output)

def sigmoid_activation(Input, Derivative = False , dE = None):
    if(Derivative):
        S = sigmoid_activation(Input)
        out = S*(1-S) * dE
        return(out)
    else:
        
        output = np.exp(Input)/(np.exp(Input)+1)
        return(output)

def neuron_forward(X,Params):
    W,B = Params
    out = np.dot(X,W) + B
    activated = sigmoid_activation(out)
    
    if(np.isnan(activated).any()):
        print("Debug logs:")
        print("X:",X)
        print("W:",W)
        print("B:",B)
        print("Out:",out)
        print("Activated:",activated)
        raise Exception("Nan detected!")
        
    return activated

def neuron_backward(dE,X,Params):
    W,B = Params
    dE = sigmoid_activation(np.dot(X,W) + B,Derivative = True,dE = dE)
    dW = np.array(dE)*X
    dB = dE
    W = W - dW
    B = B - dB
    dX = np.array(dE)*W
    return (W,B),(dX)

def add_dense_layer(N_Neurons, Model = [], input_size = None ):
    
    if(input_size==None):
        input_size = len(Model[-1])
        
    new_layer = []
    for i in range(N_Neurons):
        new_layer = new_layer + [(np.random.rand(input_size),np.random.rand())]
    Model = Model + [new_layer]
    
    return Model

def Forward_propogation(Model , X ,Activation = None):
    All_outputs = [X]
    for layer in Model:
        Layer_output = []
        for neuron in layer:
            Layer_output = Layer_output + [neuron_forward(All_outputs[-1],neuron)]
        All_outputs = All_outputs + [Layer_output]
        
    if(Activation != None):
        All_outputs = All_outputs + [Activation(All_outputs[-1])]
    else:
        All_outputs = All_outputs + [All_outputs[-1]]
        
    return All_outputs
        
def Back_propogation(Model,Outputs,Error,Activation = None):
    if(Activation != None):
        Error = Activation(Outputs[-1], Derivative = True , dE = Error)
        
    n_layers = len(Model)
    Input_errors = np.array(Error)
    
    for i in range(n_layers):
        Out_Error = 0

        for j in range(len(Model[n_layers - i -1])):
            Model[n_layers - i -1][j] , dX = neuron_backward(Input_errors[j],Outputs[n_layers - i -1],Model[n_layers - i -1][j])
            Out_Error = Out_Error + dX
        Input_errors = Out_Error.copy()
        
    return Model

def display_weights(Model):
    print("\n*********************************************")
    for i,layer in enumerate(Model):
        print("\n--------Layer {}---------".format(i+1))
        for node in layer:
            print(node)
    print("*********************************************\n")

In [191]:
path = "challenges-in-representation-learning-facial-expression-recognition-challenge/"
data = pd.read_csv(path+'icml_face_data.csv')

print("Number of emotions: ",data['emotion'].unique())
data.head()

Number of emotions:  [0 2 4 6 3 5 1]


,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [192]:
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

In [193]:
X_train,Y_train = prepare_data(data)

In [194]:
print(np.shape(X_train),np.shape(Y_train))

(35887, 48, 48) (35887,)


In [198]:
Model = add_dense_layer(7,input_size =48*48)
Model = add_dense_layer(7,Model)
Model = add_dense_layer(7,Model)
# display_weights(Model)

In [203]:
# Outputs = Forward_propogation(Model,[0.1,0.1])
# print(Outputs)
# display_weights(Model)
# Model = Back_propogation(Model,Outputs,[2,1])
# display_weights(Model)
n_classes = 7
learning_rate = 0.1
Epochs = 10
Loss = []

for i in tqdm.tqdm(range(Epochs)):
    for X,Truth in zip(X_train,Y_train):
        X = X.flatten()/255
        record = []
        #Obtain 
        Outputs = Forward_propogation(Model,X,softmax)

        #Save prediction for plotting
        record = record + [np.argmax(Outputs[-1]) == Truth]
        
        #Calculate error
        temp = [0]*n_classes
        temp[Truth] = 1
        Error = (Outputs[-1] - temp)* learning_rate
        
        display_weights(Model)
        print(Outputs)
        
        #Backpropogate using error
        Model = Back_propogation(Model,Outputs,Error,softmax)
    Loss = Loss + [np.mean(record)]
    
plt.plot(Loss)
plt.show
print(Outputs[-1])


*********************************************

--------Layer 1---------
(array([0.21124665, 0.01505739, 0.27947815, ..., 0.36104707, 0.61640066,
       0.5311541 ]), 0.8630128316114897)
(array([0.12827231, 0.53057393, 0.58784118, ..., 0.09918642, 0.02227738,
       0.08405145]), 0.04684262208899892)
(array([0.50271253, 0.09928989, 0.32871982, ..., 0.34523604, 0.81057335,
       0.43090422]), 0.22982960384409268)
(array([0.51622197, 0.11401057, 0.21770291, ..., 0.55152366, 0.27833287,
       0.0313195 ]), 0.9391868677588338)
(array([0.56462098, 0.4353721 , 0.80137654, ..., 0.35891892, 0.52081978,
       0.66782292]), 0.7533723850119036)
(array([0.85836477, 0.71659625, 0.09267893, ..., 0.3792522 , 0.01689892,
       0.68985289]), 0.12202942442940046)
(array([0.44059837, 0.37151685, 0.14367438, ..., 0.34135419, 0.91844012,
       0.93452209]), 0.4326695996489279)

--------Layer 2---------
(array([0.49157251, 0.8627586 , 0.71280596, 0.43102267, 0.02707028,
       0.99200285, 0.4268604 ]),

<ipython-input-202-7c1ffb2eb336>:20: RuntimeWarning: overflow encountered in exp
  output = np.exp(Input)/(np.exp(Input)+1)
<ipython-input-202-7c1ffb2eb336>:20: RuntimeWarning: invalid value encountered in double_scalars
  output = np.exp(Input)/(np.exp(Input)+1)


Exception: Nan detected!